# <img style="float: left; padding-right: 10px; width: 45px" src="https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/iacs.png"> CS109A Introduction to Data Science: 

## Homework 2: Linear and k-NN Regression


**Harvard University**<br/>
**Fall 2018**<br/>
**Instructors**: Pavlos Protopapas, Kevin Rader

<hr style="height:2pt">


In [1]:
#RUN THIS CELL 
import requests
from IPython.core.display import HTML
styles = requests.get("https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/cs109.css").text
HTML(styles)

### INSTRUCTIONS

- To submit your assignment follow the instructions given in canvas.
- Restart the kernel and run the whole notebook again before you submit. 
- If you submit individually and you have worked with someone, please include the name of your [one] partner below. 
- As much as possible, try and stick to the hints and functions we import at the top of the homework, as those are the ideas and tools the class supports and is aiming to teach. And if a problem specifies a particular library you're required to use that library, and possibly others from the import list.

Names of people you have worked with goes here: 

Avriel Epps

In [2]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from statsmodels.api import OLS
import seaborn as sns
from pandas.core import datetools
%matplotlib inline

/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


<div class="theme"> Predicting Taxi Pickups in NYC </div>

In this homework, we will explore k-nearest neighbor and linear regression methods for predicting a quantitative variable. Specifically, we will build regression models that can predict the number of taxi pickups in New York city at any given time of the day. These prediction models will be useful, for example, in monitoring traffic in the city.

The data set for this problem is given in the file `dataset_1.csv`.  You will need to separate it into training and test sets. The first column contains the time of a day in minutes, and the second column contains the number of pickups observed at that time. The data set covers taxi pickups recorded in NYC during Jan 2015.

We will fit regression models that use the time of the day (in minutes) as a predictor and predict the average number of taxi pickups at that time. The models will be fitted to the training set and  evaluated on the test set. The performance of the models will be evaluated using the $R^2$ metric.

<div class="exercise"> <b> Question 1  [25 pts]</b> </div>

**1.1**. Use pandas to load the dataset from the csv file `dataset_1.csv` into a pandas data frame.  Use the `train_test_split` method from `sklearn` with a `random_state` of 42 and a `test_size` of 0.2 to split the dataset into training and test sets.  Store your train set dataframe in the variable `train_data`.  Store your test set dataframe in the variable `test_data`.


**1.2**. Generate a scatter plot of the training data points with clear labels on the x and y axes. The time of the day on the x-axis and the number of taxi pickups on the y-axis.  Make sure to title your plot.

**1.3**. Does the pattern of taxi pickups make intuitive sense to you? 


### Answers

**1.1 Use pandas to load the dataset from the csv file ...**

In [3]:
# read the file
data = pd.read_csv('data/dataset_1.csv', sep=",")


In [4]:
print(data.columns)

Index(['TimeMin', 'PickupCount'], dtype='object')


In [6]:
#Check that it imported correctly -- delete before turning in
display(data.head())

,TimeMin,PickupCount
0,860.0,33.0
1,17.0,75.0
2,486.0,13.0
3,300.0,5.0
4,385.0,10.0


In [ ]:
#data['Time (TimeMin)'].astype(str)
#data['Pick Up Count'].astype(str)

In [ ]:
# split the data
train_data, test_data = train_test_split(data, test_size = 0.2, random_state = 42)

In [ ]:
x_train, y_train = train_data['TimeMin'], train_data['PickupCount']
x_test, y_test = test_data['TimeMin'], test_data['PickupCount']

In [ ]:
# Test size is indeed 20% of total
data20 = .2*len(data)
our20 = len(test_data)
print("20% of data:",data20,"Our split data:", our20)

**1.2 Generate a scatter plot of the training data points**

In [ ]:
y_train.dtype

In [ ]:
x_train.dtype

In [ ]:
#sns.relplot(x="Time of Day", y="Number of Pickups", data=train_data)

In [ ]:
def nice_scatterplot(x, y, title):
    # font size
    f_size = 18
    
    # make the figure
    fig, ax = plt.subplots(1,1, figsize=(8,5)) # Create figure object

    # set axes limits to make the scale nice
    ax.set_xlim(np.min(x)-1, np.max(x) + 1)
    ax.set_ylim(np.min(y)-1, np.max(y) + 1)

    # adjust size of tickmarks in axes
    ax.tick_params(labelsize = f_size)
    
    # remove tick labels
    ax.tick_params(labelbottom=False,  bottom=False)
    
    # adjust size of axis label
    ax.set_xlabel(r'$x$', fontsize = f_size)
    ax.set_ylabel(r'$y$', fontsize = f_size)
    
    # set figure title label
    ax.set_title(title, fontsize = f_size)

    # you may set up grid with this 
    ax.grid(True, lw=1.75, ls='--', alpha=0.15)

    # make actual plot (Notice the label argument!)
    #ax.scatter(x, y, label=r'$my points$')
    #ax.scatter(x, y, label='$my points$')
    ax.scatter(x, y, label=r'$my\,points$')
    ax.legend(loc='best', fontsize = f_size);
    
    return ax

In [ ]:
nice_scatterplot(data["TimeMin"],data["PickupCount"],"Time of Day vs. Taxi Pick Ups")
plt.xlabel("Time of Day")
plt.ylabel("Number of Taxi Pick Ups")

In [ ]:
sns.set_context("paper")
plt.scatter(x_train,y_train)
plt.title('Predicting Taxi Pick-Ups in New York')
plt.xlabel("Time of Day in Minutes")
plt.ylabel("Number of Taxi Pick Ups")

**1.3 Discuss your results. Does the pattern of taxi pickups make intuitive sense to you?**

*your answer here*


<div class="exercise"> <b>Question 2 [25 pts]</b> </div>

In lecture we've seen k-Nearest Neighbors (k-NN) Regression, a non-parametric regression technique.  In the following problems please use built in functionality from `sklearn` to run k-NN Regression. 


**2.1**. Choose `TimeMin` as your feature variable and `PickupCount` as your response variable.  Create a dictionary of `KNeighborsRegressor` objects and call it `KNNModels`.  Let the key for your `KNNmodels` dictionary be the value of $k$ and the value be the corresponding `KNeighborsRegressor` object. For $k \in \{1, 10, 75, 250, 500, 750, 1000\}$, fit k-NN regressor models on the training set (`train_data`). 

**2.2**.  For each $k$ on the training set, overlay a scatter plot of the actual values of `PickupCount` vs. `TimeMin` with a scatter plot of **predictions** for `PickupCount` vs  `TimeMin`.  Do the same for the test set.  You should have one figure with 2 x 7 total subplots; for each $k$ the figure should have two subplots, one subplot for the training set and one for the test set. 

**Hints**:
1. Each subplot should use different color and/or markers to distinguish k-NN regression prediction values from the actual data values.
2. Each subplot must have appropriate axis labels, title, and legend.
3. The overall figure should have a title.  


**2.3**. Report the $R^2$ score for the fitted models on both the training and test sets for each $k$ (reporting the values in tabular form is encouraged).

**2.4**. Plot, in a single figure, the $R^2$ values from the model on the training and test set as a function of $k$.  

**Hints**:
1. Again, the figure must have axis labels and a legend.
2. Differentiate $R^2$ visualization on the training and test set by color and/or marker.
3. Make sure the $k$ values are sorted before making your plot.

**2.5**. Discuss the results:

1. If $n$ is the number of observations in the training set, what can you say about a k-NN regression model that uses $k = n$?  
2. What does an $R^2$ score of $0$ mean?  
3. What would a negative $R^2$ score mean?  Are any of the calculated $R^2$ you observe negative?
4. Do the training and test $R^2$ plots exhibit different trends?  Describe.  
5. How does the value of $k$ affect the fitted model and in particular the training and test $R^2$ values?
6. What is the best value of $k$ and what are the corresponding training/test set $R^2$ values?

### Answers

**2.1 Choose `TimeMin` as your feature variable and `PickupCount` as your response variable.  Create a dictionary ... **

In [ ]:
# Set kNN parameter:
k = 10

# Now we can fit the model, predict our variable of interest, and then evaluate our fit:
# First, we create the classifier object:
neighbors = KNeighborsRegressor(n_neighbors=k)

# Then, we fit the model using x_train as training data and y_train as target values:
neighbors.fit(train_data[['TimeMin']], train_data['PickupCount'])

neighbors

# Retreieve our predictions:
prediction_knn = neighbors.predict(test_data[['TimeMin']])

#put this in a function
def get_knn(k):
    neighbors = KNeighborsRegressor(n_neighbors=k)
    neighbors.fit(train_data[['TimeMin']], train_data['PickupCount'])
    return neighbors

In [ ]:
KNNModels = {}
KNNModels['1']=get_knn(k=1)
KNNModels['10']=get_knn(k=10)
KNNModels['75']=get_knn(k=75)
KNNModels['250']=get_knn(k=250)
KNNModels['500']=get_knn(k=500)
KNNModels['750']=get_knn(k=750)
KNNModels['1000']=get_knn(k=1000)

KNNModels

**2.2 For each $k$ on the training set, overlay a scatter plot ... **

In [ ]:
for k in (1, 10, 75, 250, 500, 750, 1000):
    neighbors = get_knn(k)

    # Then, we fit the model using x_train as training data and y_train as target values:
    neighbors.fit(train_data[['TimeMin']], train_data['PickupCount'])

    # Retreieve our predictions:
    prediction_knn = neighbors.predict(test_data[['TimeMin']])

    #plot
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(20,6))
    axes[0].plot(train_data['TimeMin'], train_data['PickupCount'], 'o', label = 'Data' )
    axes[0].plot(train_data['TimeMin'], neighbors.predict(train_data[['TimeMin']]), '*', label = 'Prediction')
    axes[0].set_xlabel('Time of Day in Minutes')
    axes[0].set_ylabel('# of Pickups')
    axes[0].set_title("Training Set")
    axes[0].legend()

    axes[1].plot(test_data['TimeMin'], test_data['PickupCount'],'o', label = 'Data' )
    axes[1].plot(test_data['TimeMin'], prediction_knn, '*', label = 'Prediction')
    axes[1].set_xlabel('Time of Day in Minutes')
    axes[1].set_ylabel('# of Pickups')
    axes[1].set_title("Test Set")
    axes[1].legend()

    fig.suptitle(f"Taxi Pickups in New York When k ={k}");

**2.3 Report the $R^2$ score for the fitted models ... **

In [ ]:
def get_r_2(k):
    array_train, array_test = [],[]
    for k in (1, 10, 75, 250, 500, 750, 1000):
        neighbors = get_knn(k)
        r2_train = neighbors.score(train_data[['TimeMin']], train_data['PickupCount'])
        r2_test = neighbors.score(test_data[['TimeMin']], test_data['PickupCount'])
        print("Length of Test Data:", len(test_data['PickupCount']))
        print("k:", k)
        print("R^2 Score of kNN on test set:", r2_test)
        print("R^2 Score of kNN on training set:", r2_train)
        print("\n")
        array_train.append(r2_train)
        array_test.append(r2_test)
    return array_train, array_test
    
    
array_test, array_train = get_r_2(k)
    

**2.4 Plot, in a single figure, the $R^2$ values from the model on the training and test set as a function of $k$**

In [ ]:
ks = [1, 10, 75, 250, 500, 750, 1000]
fig, ax = plt.subplots(1,1, figsize=(15,9))
ax.plot(ks, array_train,'-o', label = 'Train R^2 Values' )
ax.plot(ks, array_test,'-*', label = 'Test R^2 Values' )
ax.set_xlabel(r'$k$')
ax.set_ylim([-0.5, 1])
ax.set_ylabel(r'$R^{2}$')
ax.legend()

**2.5 Discuss the results**

1. If $n$ is the number of observations in the training set, what can you say about a k-NN regression model that uses $k = n$?  
2. What does an $R^2$ score of $0$ mean?  
3. What would a negative $R^2$ score mean?  Are any of the calculated $R^2$ you observe negative?
4. Do the training and test $R^2$ plots exhibit different trends?  Describe.  
5. How does the value of $k$ affect the fitted model and in particular the training and test $R^2$ values?
6. What is the best value of $k$ and what are the corresponding training/test set $R^2$ values?

<div class="exercise"> <b> Question 3 [25 pts] </b></div>

We next consider simple linear regression, which we know from lecture is a parametric approach for regression that assumes that the response variable has a linear relationship with the predictor.  Use the `statsmodels` module for Linear Regression. This module has built-in functions to summarize the results of regression and to compute confidence intervals for estimated regression parameters.  

**3.1**. Again choose `TimeMin` as your predictor and `PickupCount` as your response variable.  Create a `OLS` class instance and use it to fit a Linear Regression model on the training set (`train_data`).  Store your fitted model in the variable `OLSModel`.

**3.2**. Re-create your plot from 2.2 using the predictions from `OLSModel` on the training and test set. You should have one figure with two subplots, one subplot for the training set and one for the test set.

**Hints**:
1. Each subplot should use different color and/or markers to distinguish Linear Regression prediction values from that of the actual data values.
2. Each subplot must have appropriate axis labels, title, and legend.
3. The overall figure should have a title.  


**3.3**. Report the $R^2$ score for the fitted model on both the training and test sets.

**3.4**. Report the slope and intercept values for the fitted linear model.  

**3.5**. Report the $95\%$ confidence interval for the slope and intercept.

**3.6**. Create a scatter plot of the residuals ($e = y - \hat{y}$) of the linear regression model on the training set as a function of the predictor variable (i.e. `TimeMin`). Place on your plot a horizontal line denoting the constant zero residual.  

**3.7**. Discuss the results:

1. How does the test $R^2$ score compare with the best test $R^2$ value obtained with k-NN regression?
2. What does the sign of the slope of the fitted linear model convey about the data?  
3. Based on the $95\%$ confidence interval, do you consider the estimates of the model parameters to be reliable?  
4. Do you expect $99\%$ confidence intervals for the slope and intercept to be tighter or wider than the $95\%$ confidence intervals? Briefly explain your answer.  
5. Based on the residuals plot that you made, discuss whether or not the assumption of linearity is valid for this data.
6. Based on the data structure, what restriction on the model would you put at the endpoints (at $x\approx0$ and $x\approx1440$)?   What does this say about the linearity assumption?


### Answers

**3.1 Again choose `TimeMin` as your predictor and `PickupCount` as your response variable.  Create a `OLS` class instance ... **

In [ ]:
#Add constant
x_ca = sm.add_constant(data['TimeMin'])
x_train_ca = sm.add_constant(x_train)
x_test_ca = sm.add_constant(x_test)

#We must first create the linear regression object from stats model
OLSmodel = sm.OLS(y_train, x_train_ca)
results = OLSmodel.fit()
display(results.summary())

#predict y
predicted_y = results.predict(x_ca)
predicted_y_train = results.predict(x_train_ca)
predicted_y_test = results.predict(x_test_ca)

**3.2 Re-create your plot from 2.2 using the predictions from `OLSModel` on the training and test set ... **

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(20,6))
axes[0].plot(train_data['TimeMin'], train_data['PickupCount'],'o', label = 'Data' )
axes[0].plot(train_data['TimeMin'], predicted_y_train, '-*', label = 'Prediction')
axes[0].set_xlabel('Time of Day in Minutes')
axes[0].set_ylabel('# of Pickups')
axes[0].set_title("Train Set")
axes[0].legend()

axes[1].plot(test_data['TimeMin'], test_data['PickupCount'], 'o', label = 'Test Data' )
axes[1].plot(test_data['TimeMin'], predicted_y_test, '-*', label = 'Prediction')
axes[1].set_xlabel('Time of Day in Minutes')
axes[1].set_ylabel('# of Pickups')
axes[1].set_title("Test Set")
axes[1].legend()

fig.suptitle("OLS Predictions for Taxi Pickups in New York");

**3.3 Report the $R^2$ score for the fitted model on both the training and test sets. **

In [ ]:
print("R^2 Score for Linear Regression (Training):", (results.rsquared))
print("R^2 Score for Linear Regression (Testing) :", (r2_score(test_data['PickupCount'], predicted_y_test)))

**3.4 Report the slope and intercept values for the fitted linear model.  **

In [ ]:
results = OLSmodel.fit()
display(results.params)

#or

beta0_sm = results.params[0]
beta1_sm = results.params[1]

print("The intercept value is {0:8.6f}, and the slope value is {1:8.6f}".format(beta0_sm, beta1_sm))

**3.5 Report the $95\%$ confidence interval for the slope and intercept.**

In [ ]:
thresh = 0.05
intervals = results.conf_int(alpha=thresh)
intervals = intervals.rename(index=str, columns={0:str(thresh/2*100)+"%",1:str((1-thresh/2)*100)+"%"})
display(intervals)

**3.6 Create a scatter plot of the residuals**

In [ ]:
residual = data['PickupCount']-predicted_y
display(residual.head())

In [ ]:
sns.set(style="whitegrid")
plt.scatter(data['TimeMin'], residual)
plt.xlabel("X1 - a predictor")
plt.ylabel("residual")
plt.show()

**3.7 Discuss the results:**

1. How does the test $R^2$ score compare with the best test $R^2$ value obtained with k-NN regression?
2. What does the sign of the slope of the fitted linear model convey about the data?  
3. Based on the $95\%$ confidence interval, do you consider the estimates of the model parameters to be reliable?  
4. Do you expect $99\%$ confidence intervals for the slope and intercept to be tighter or wider than the $95\%$ confidence intervals? Briefly explain your answer.  
5. Based on the residuals plot that you made, discuss whether or not the assumption of linearity is valid for this data.
6. Based on the data structure, what restriction on the model would you put at the endpoints (at $x\approx0$ and $x\approx1440$)?   What does this say about the linearity assumption?

<div class="theme"> Outliers </div>

You may recall from lectures that OLS Linear Regression can be susceptible to outliers in the data.  We're going to look at a dataset that includes some outliers and get a sense for how that affects modeling data with Linear Regression.  **Note, this is an open-ended question, there is not one correct solution (or one correct definition of an outlier).**


<div class="exercise"><b> Question 4 [25 pts] </b></div>




**4.1**. We've provided you with two files `outliers_train.txt` and `outliers_test.txt` corresponding to training set and test set data.  What does a visual inspection of training set tell you about the existence of outliers in the data?

**4.2**. Choose `X` as your feature variable and `Y` as your response variable.  Use `statsmodel` to create a Linear Regression model on the training set data.  Store your model in the variable `OutlierOLSModel`.

**4.3**. You're given the knowledge ahead of time that there are 3 outliers in the training set data.  The test set data doesn't have any outliers.  You want to remove the 3 outliers in order to get the optimal intercept and slope.  In the case that you're sure of the existence and number (3) of outliers ahead of time, one potential brute force method to outlier detection might be to find the best Linear Regression model on all possible subsets of the training set data with 3 points removed.  Using this method, how many times will you have to calculate the Linear Regression coefficients on the training data?

**4.4**  In CS109 we're strong believers that creating heuristic models is a great way to build intuition. In that spirit, construct an approximate algorithm to find the 3 outlier candidates in the training data by taking advantage of the Linear Regression residuals. Place your algorithm in the function `find_outliers_simple`.  It should take the parameters `dataset_x` and `dataset_y` representing your features and response variable values (make sure your response variable is stored as a numpy column vector).  The return value should be a list `outlier_indices` representing the indices of the 3 outliers in the original datasets you passed in.  Remove the outliers that your algorithm identified, use `statsmodels` to create a Linear Regression model on the remaining training set data, and store your model in the variable `OutlierFreeSimpleModel`.


**4.5** Create a figure with two subplots: the first is a scatterplot where the color of the points denotes the outliers from the non-outliers in the training set, and include two regression lines on this scatterplot: one fitted with the outliers included and one fitted with the outlier removed (all on the training set).  The second plot should include a scatterplot of points from the test set with the same two regression lines fitted on the training set: with and without outliers.  Visually which model fits the test set data more closely?



**4.6**. Calculate the $R^2$ score for the `OutlierOLSModel` and the `OutlierFreeSimpleModel` on the test set data.  Which model produces a better $R^2$ score?

**4.7**. One potential problem with the brute force outlier detection approach in 4.3 and the heuristic algorithm you constructed 4.4 is that they assume prior knowledge of the number of outliers.  In general you can't expect to know ahead of time the number of outliers in your dataset.  Alter the algorithm you constructed in 4.4 to create a more general heuristic (i.e. one which doesn't presuppose the number of outliers) for finding outliers in your dataset.  Store your algorithm in the function `find_outliers_general`.  It should take the parameters `dataset_x` and `dataset_y` representing your features and response variable values (make sure your response variable is stored as a numpy column vector).  It can take additional parameters as long as they have default values set.  The return value should be the list `outlier_indices` representing the indices of the outliers in the original datasets you passed in (in the order of 'severity').  Remove the outliers that your algorithm identified, use `statsmodels` to create a Linear Regression model on the remaining training set data, and store your model in the variable `OutlierFreeGeneralModel`.

**Hints**:
   1. How many outliers should you try to identify in each step?
   2. If you plotted an $R^2$ score for each step the algorithm, what might that plot tell you about stopping conditions?
    
**4.8**. Run your algorithm in 4.7 on the training set data.  
1. What outliers does it identify?
2. How do those outliers compare to the outliers you found in 4.4?
3. How does the general outlier-free Linear Regression model you created in 4.7 perform compared to the simple one in 4.4?

### Answers
**4.1  We've provided you with two files `outliers_train.txt` and `outliers_test.txt` corresponding to training set and test set data.  What does a visual inspection of training set tell you about the existence of outliers in the data? ** 

In [ ]:
# read the data
# your code here
train_data = pd.read_csv('data/outliers_train.csv', sep=",")
test_data = pd.read_csv('data/outliers_train.csv', sep=",")

In [ ]:
# your code here
print(test_data)

In [ ]:
# your code here
def nice_scatterplot(x, y, title):
    # font size
    f_size = 18
    
    # make the figure
    fig, ax = plt.subplots(1,1, figsize=(8,5)) # Create figure object

    # set axes limits to make the scale nice
    ax.set_xlim(np.min(x)-1, np.max(x) + 1)
    ax.set_ylim(np.min(y)-1, np.max(y) + 1)

    # adjust size of tickmarks in axes
    ax.tick_params(labelsize = f_size)
    
    # remove tick labels
    ax.tick_params(labelbottom=False,  bottom=False)
    
    # adjust size of axis label
    ax.set_xlabel(r'$x$', fontsize = f_size)
    ax.set_ylabel(r'$y$', fontsize = f_size)
    
    # set figure title label
    ax.set_title(title, fontsize = f_size)

    # you may set up grid with this 
    ax.grid(True, lw=1.75, ls='--', alpha=0.15)

    # make actual plot (Notice the label argument!)
    #ax.scatter(x, y, label=r'$my points$')
    #ax.scatter(x, y, label='$my points$')
    ax.scatter(x, y, label=r'$my\,points$')
    ax.legend(loc='best', fontsize = f_size);
    
    return ax

In [ ]:
# your code here
nice_scatterplot(train_data["X"],train_data["Y"],"Outliers: X v. Y")
plt.xlabel("X")
plt.ylabel("Y")

In [ ]:
sns.set_context("paper")
plt.scatter(train_data["X"],train_data["Y"])
plt.title('Predicting Taxi Pick-Ups in New York')
plt.xlabel("Time of Day in Minutes")
plt.ylabel("Number of Taxi Pick Ups")

*your answer here*


**4.2 Choose `X` as your feature variable and `Y` as your response variable.  Use `statsmodel` to create ... **

In [ ]:
# your code here


**4.3 One potential brute force method to outlier detection might be to find the best Linear Regression model on all possible subsets of the training set data with 3 points removed.  Using this method, how many times will you have to calculate the Linear Regression coefficients on the training data?**
  
  

*your answer here*


**4.4 CS109 hack ... **

In [ ]:
def find_outliers_simple(dataset_x, dataset_y):
    # your code here


In [ ]:
# get outliers
# your code here


In [ ]:
simple_outliers_y

In [ ]:
# calculate outlier model
# your code here


**4.5 Create a figure with two subplots: the first is a scatterplot ... **

In [ ]:
# plot 
# your code here


*your answer here*


**4.6  Calculate the $R^2$ score for the `OutlierOLSModel` and the `OutlierFreeSimpleModel` on the test set data.  Which model produces a better $R^2$ score?**

In [ ]:
# your code here


**4.7 One potential problem with the brute force outlier detection approach in 4.3 and the heuristic algorithm you constructed 4.4 is that they assume prior knowledge of the number of outliers. **

In [ ]:
# your code here


**4.8 Run your algorithm in 4.7 on the training set data **

In [ ]:
# your code here


In [ ]:
# your code here


In [ ]:
# your code here


In [ ]:
# your code here


*your answer here*
